In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from data_manager import *
from metadata import *
from conversion import images_to_volume

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt


In [2]:
for stack in all_nissl_stacks:
# for stack in ['MD591']:
    first_sec, last_sec = metadata_cache['section_limits'][stack]
#     bar = show_progress_bar(first_sec, last_sec)

    images = {}
    for sec in range(first_sec, last_sec):
#         bar.value = sec
        try:
            fn = DataManager.get_image_filepath(stack, section=sec, resol='thumbnail', version='cropped_tif')
            img = img_as_ubyte(rgb2gray(imread(fn)))

            # apply foreground mask
            mask = DataManager.load_thumbnail_mask_v2(stack=stack, section=sec, version='aligned_cropped')
            img[~mask] = 0

            images[sec-1] = img
        except:
            pass

    downsample = 32
    voxel_z_size = SECTION_THICKNESS/(XY_PIXEL_DISTANCE_LOSSLESS * downsample)
    
    volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=last_sec-1)
    
#     basename = DataManager.get_original_volume_basename(stack=stack, volume_type='intensity')
#     vol_fp = os.path.join(VOLUME_ROOTDIR, stack, basename, basename + '.bp')
    vol_fp = DataManager.get_intensity_volume_filepath(stack=stack, downscale=downsample)
    create_if_not_exists(os.path.dirname(vol_fp))
    bp.pack_ndarray_file(volume, vol_fp)
    
#     bbox_fp = os.path.join(VOLUME_ROOTDIR, stack, basename, basename + '_bbox.txt')
    bbox_fp = DataManager.get_intensity_volume_bbox_filepath(stack=stack, downscale=downsample)
    np.savetxt(bbox_fp, np.array(volume_bbox, np.int)[None, :], fmt='%d')

/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
